In [3]:
#4company_combo_oldest_day_qty

import pandas as pd
from itertools import combinations

# ------------------ USER INPUTS ------------------
input_file = r"D:\Trading Stratergies\Stratergy 2 All Codes\50 comp data (input file).xlsx"
INVESTMENT = float(input("Enter Investment amount per company (example 50000): "))
output_file = r"C:\Users\Swarupa\Downloads\4company_combo_qty.xlsx"

# ------------------ LOAD DATA ------------------
df = pd.read_excel(input_file)
company_names = df.columns[1:].tolist()
df['DATE'] = pd.to_datetime(df['DATE'], dayfirst=True)

# ------------------ SKIP FIRST 3 MONTHS ------------------
first_date = df['DATE'].min()
cutoff_date = first_date + pd.DateOffset(months=3)

df_filtered = df[df['DATE'] > cutoff_date].copy()

print("Full dataset start date:", first_date)
print("Cutoff date after skipping 3 months:", cutoff_date)

# ------------------ TAKE NEXT DATE AFTER SKIP ------------------
selected_date = df_filtered['DATE'].min()   
prices_row = df_filtered[df_filtered['DATE'] == selected_date].iloc[0]

print("Selected date for quantity calculation:", selected_date)

# ------------------ CALCULATE QUANTITY ------------------
rows = []

for combo in combinations(company_names, 4):
    combo_name = ",".join(combo)
    qty_list = []

    for company in combo:
        price = prices_row[company]

        if price == 0 or pd.isna(price):
            qty_list.append("NA")
        else:
            qty_list.append(f"{INVESTMENT / price:.2f}")

    rows.append([combo_name, ",".join(qty_list)])

# ------------------ SAVE OUTPUT ------------------
out_df = pd.DataFrame(rows, columns=['Combo', 'Quantity'])
out_df.to_excel(output_file, index=False)

print("\nDone! File saved at:", output_file)


Enter Investment amount per company (example 50000): 50000
Full dataset start date: 2020-01-01 00:00:00
Cutoff date after skipping 3 months: 2020-04-01 00:00:00
Selected date for quantity calculation: 2020-04-03 00:00:00

Done! File saved at: C:\Users\Swarupa\Downloads\4company_combo_qty.xlsx


In [7]:
# 4Combo_TotalPrices

import pandas as pd
import numpy as np
from tqdm import tqdm
import csv
from datetime import timedelta

# ---------------- INPUT PATHS ----------------
file_prices = r"D:\Trading Stratergies\Stratergy 2 All Codes\50 comp data (input file).xlsx"
file_combos = r"D:\Trading Stratergies\Backtesting\4company_combo_qty.xlsx"
output_file = r"D:\Trading Stratergies\Backtesting\4company_combo_total.csv"

# ---------------- LOAD PRICE DATA ----------------
df_price = pd.read_excel(file_prices)
date_col = df_price.columns[0]
company_cols = df_price.columns[1:]

df_price[date_col] = pd.to_datetime(df_price[date_col], dayfirst=True)

# ---------------- SKIP FIRST 3 MONTHS ----------------
first_date = df_price[date_col].min()
cutoff_date = first_date + pd.DateOffset(months=3)

df_filtered = df_price[df_price[date_col] > cutoff_date].copy()

print("Full dataset start date:", first_date)
print("Cutoff date after skipping first 3 months:", cutoff_date)

# ---------------- TAKE NEXT AVAILABLE DATE ----------------
start_date = df_filtered[date_col].min()
print("Start date for calculation =", start_date)

# ---------------- 45 CALENDAR DAYS WINDOW ----------------
end_date = start_date + timedelta(days=44)   

df_window = df_filtered[
    (df_filtered[date_col] >= start_date) &
    (df_filtered[date_col] <= end_date)
].copy()

print("45-day window start:", df_window[date_col].min())
print("45-day window end:", df_window[date_col].max())

# ---------------- LOAD COMBOS ----------------
df_combo = pd.read_excel(file_combos)
df_combo.columns = df_combo.columns.str.strip()

df_combo[['C1','C2','C3','C4']] = df_combo['Combo'].str.split(',', expand=True)

# Convert qty string → float list
def split_qty(qty):
    out = []
    for x in str(qty).split(","):
        try: out.append(float(x.strip()))
        except: out.append(0.0)
    while len(out) < 4:
        out.append(0.0)
    return out[:4]

df_combo[['Q1','Q2','Q3','Q4']] = df_combo['Quantity'].apply(split_qty).tolist()

# Create mapping of column index
col_index = {col: i for i, col in enumerate(company_cols)}

combo_indices = df_combo[['C1','C2','C3','C4']].applymap(
    lambda x: col_index.get(x, -1)
).values

qty_matrix = df_combo[['Q1','Q2','Q3','Q4']].values

num_dates = len(df_window)
num_combos = len(df_combo)

# ---------------- SAVE CSV HEADER ----------------
with open(output_file, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["DATE", "Combo", "Price_Combo", "TotalPrice"])

# ---------------- PROCESS ----------------
with open(output_file, "a", newline="") as f:
    writer = csv.writer(f)
    pbar = tqdm(total=num_dates * num_combos, desc="Processing", unit="rows")

    for idx, row in df_window.iterrows():
        date_value = row[date_col]
        price_row = row[company_cols].to_numpy(dtype=float)

        for i in range(num_combos):
            combo_idx = combo_indices[i]
            qty = qty_matrix[i]

            # Extract prices for this combo
            prices = price_row[combo_idx]
            prices = np.where(combo_idx == -1, 0.0, prices)

            total_price = np.sum(prices * qty)

            writer.writerow([
                date_value,
                df_combo["Combo"].iloc[i],
                list(prices),
                total_price
            ])

            pbar.update(1)

    pbar.close()

print("\nOutput generated at:")
print(output_file)


Full dataset start date: 2020-01-01 00:00:00
Cutoff date after skipping first 3 months: 2020-04-01 00:00:00
Start date for calculation = 2020-04-03 00:00:00
45-day window start: 2020-04-03 00:00:00
45-day window end: 2020-05-15 00:00:00


C:\Users\Swarupa\AppData\Local\Temp\4\ipykernel_5692\3766591394.py:66: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  combo_indices = df_combo[['C1','C2','C3','C4']].applymap(

Processing:   3%|█▉                                                       | 217204/6218100 [01:04<29:42, 3367.08rows/s]

Processing:   4%|██▏                                                     | 240004/6218100 [00:13<05:23, 18452.67rows/s]


Processing:   8%|████▍                                                   | 486165/6218100 [00:26<05:11, 18429.73rows/s]


Processing:  12%|██████▌                                                 | 731568/6218100 [00:39<04:56, 18478.52rows/s]


Processing:  16%|████████▊                                               | 981386/6218100 [00:53<04:43, 18478.28rows/s]


...

Processing:  88%|████████████████████████████████████████████████▋      | 5502485/6218100 [04:56<00:38, 18732.27rows/s]


Processing:  92%|██████████████████████████████████████████████████▊    | 5747613/6218100 [05:10<00:25, 18348.74rows/s]


Processing:  96%|█████████████████████████████████████████████████████  | 5992385/6218100 [05:23<00:12, 18373.09rows/s]


Processing: 100%|███████████████████████████████████████████████████████| 6218100/6218100 [05:35<00:00, 18522.68rows/s]


Output generated at:
D:\Trading Stratergies\Backtesting\4company_combo_total.csv


In [9]:
# 4Company_EURINR_Ratio_50DMA

import pandas as pd
import numpy as np
from tqdm import tqdm


# INPUT FILES
file_csv = r"D:\Trading Stratergies\Stratergy 2 All Codes\4company_combo_total_price.csv"
file_eurinr = r"D:\Trading Stratergies\Stratergy 2 All Codes\EURINR_Values(input file).xlsx"
final_output = r"D:\Trading Stratergies\Backtesting\50DMA_EURINR.csv"


# LOAD EURINR
eur = pd.read_excel(file_eurinr, usecols=["DATE", "EURINR"])
eur["DATE"] = pd.to_datetime(eur["DATE"])
eur = eur.sort_values("DATE") 


# FIXED STARTING DATE + 45 CALENDAR DAYS
current_oldest_date = pd.to_datetime("2020-04-03")
calendar_end_date = current_oldest_date + pd.Timedelta(days=44)

eur_window = eur[eur["DATE"] <= calendar_end_date].copy()
eur_dict = dict(zip(eur_window["DATE"].values, eur_window["EURINR"].values))
valid_dates = set([d for d in eur_window["DATE"].values if d >= current_oldest_date])

print(f"Processing EURINR from {current_oldest_date.date()} to {calendar_end_date.date()} "
      f"({len(valid_dates)} trading days available)")


# STREAMING PARAMETERS
chunksize = 50000
state = {} 
header_written = False
use_columns = ["DATE", "Combo", "TotalPrice"]


# STREAM CSV AND PROCESS
for chunk in tqdm(pd.read_csv(file_csv,
                              chunksize=chunksize,
                              parse_dates=["DATE"],
                              usecols=use_columns),
                  desc="Processing CSV", unit="rows"):

    # Keep only rows in 45-calendar-day window
    chunk = chunk[chunk["DATE"].isin(valid_dates)]
    if chunk.empty:
        continue

    # Map EURINR and compute ratio
    date_values = chunk["DATE"].values
    total_prices = chunk["TotalPrice"].values
    eur_values = np.array([eur_dict[d] for d in date_values])
    ratios = total_prices / eur_values

    combos = chunk["Combo"].values
    dma_result = np.full(len(chunk), np.nan, dtype=np.float64)

    # Process each combo
    unique_combos = np.unique(combos)
    for combo in unique_combos:
        combo_idx = np.where(combos == combo)[0]
        combo_ratios = ratios[combo_idx]

        # Get past ratios for continuity
        past = state.get(combo, np.array([], dtype=np.float64))
        combined = np.concatenate((past, combo_ratios))

        # Compute 50DMA
        cumsum = np.cumsum(combined, dtype=float)
        roll = np.empty(len(combined))
        for i in range(len(combined)):
            if i < 49:
                # cumulative average until 50 values
                roll[i] = cumsum[i] / (i + 1)
            else:
                # standard 50DMA
                roll[i] = (cumsum[i] - cumsum[i - 50]) / 50

        dma_vals = roll[-len(combo_ratios):]
        dma_result[combo_idx] = dma_vals

        # Keep last 49 values for continuity in next chunk
        state[combo] = combined[-49:]

    # Save required columns
    out_chunk = pd.DataFrame({
        "DATE": chunk["DATE"].values,
        "Combo": combos,
        "EURINR_Ratio": ratios,
        "50DMA_EURINR": dma_result
    })

    out_chunk.to_csv(final_output, mode="a", index=False, header=not header_written)
    header_written = True

print("\nDONE — Output saved to:")
print(final_output)


Processing EURINR from 2020-04-03 to 2020-05-17 (27 trading days available)


Processing CSV: 1643rows [02:28, 11.08rows/s]


DONE — Output saved to:
D:\Trading Stratergies\Backtesting\50DMA_EURINR.csv


In [20]:
# 4Company_NIFTY_Ratio_50DMA

import pandas as pd
import numpy as np
from tqdm import tqdm


# INPUT FILES
file_csv = r"D:\Trading Stratergies\Stratergy 2 All Codes\4company_combo_total_price.csv"
file_nifty = r"D:\Trading Stratergies\Stratergy 2 All Codes\Nifty_Index_values(input file).xlsx"
final_output = r"D:\Trading Stratergies\Backtesting\50DMA_NIFTY.csv"


# LOAD NIFTY
nifty = pd.read_excel(file_nifty, usecols=["DATE", "NIFTY"])
nifty["DATE"] = pd.to_datetime(nifty["DATE"])
nifty = nifty.sort_values("DATE") 


# FIXED STARTING DATE + 45 CALENDAR DAYS
current_oldest_date = pd.to_datetime("2020-04-03")
calendar_end_date = current_oldest_date + pd.Timedelta(days=44)

nifty_window = nifty[nifty["DATE"] <= calendar_end_date].copy()
nifty_dict = dict(zip(nifty_window["DATE"].values, nifty_window["NIFTY"].values))
valid_dates = set([d for d in nifty_window["DATE"].values if d >= current_oldest_date])

print(f"Processing NIFTY from {current_oldest_date.date()} to {calendar_end_date.date()} "
      f"({len(valid_dates)} trading days available)")


# STREAMING PARAMETERS
chunksize = 50000
state = {} 
header_written = False
use_columns = ["DATE", "Combo", "TotalPrice"]


# STREAM CSV AND PROCESS
for chunk in tqdm(pd.read_csv(file_csv,
                              chunksize=chunksize,
                              parse_dates=["DATE"],
                              usecols=use_columns),
                  desc="Processing CSV", unit="rows"):

    # Keep only rows in 45-calendar-day window
    chunk = chunk[chunk["DATE"].isin(valid_dates)]
    if chunk.empty:
        continue

    # Map NIFTY and compute ratio
    date_values = chunk["DATE"].values
    total_prices = chunk["TotalPrice"].values
    nifty_values = np.array([nifty_dict[d] for d in date_values])
    ratios = total_prices / nifty_values

    combos = chunk["Combo"].values
    dma_result = np.full(len(chunk), np.nan, dtype=np.float64)

    # Process each combo
    unique_combos = np.unique(combos)
    for combo in unique_combos:
        combo_idx = np.where(combos == combo)[0]
        combo_ratios = ratios[combo_idx]

        # Get past ratios for continuity
        past = state.get(combo, np.array([], dtype=np.float64))
        combined = np.concatenate((past, combo_ratios))

        # Compute 50DMA
        cumsum = np.cumsum(combined, dtype=float)
        roll = np.empty(len(combined))
        for i in range(len(combined)):
            if i < 49:
                # cumulative average until 50 values
                roll[i] = cumsum[i] / (i + 1)
            else:
                # standard 50DMA
                roll[i] = (cumsum[i] - cumsum[i - 50]) / 50

        dma_vals = roll[-len(combo_ratios):]
        dma_result[combo_idx] = dma_vals

        # Keep last 49 values for continuity in next chunk
        state[combo] = combined[-49:]

    # Save required columns
    out_chunk = pd.DataFrame({
        "DATE": chunk["DATE"].values,
        "Combo": combos,
        "NIFTY_Ratio": ratios,
        "50DMA_NIFTY": dma_result
    })

    out_chunk.to_csv(final_output, mode="a", index=False, header=not header_written)
    header_written = True

print("\nDONE — Output saved to:")
print(final_output)


Processing NIFTY from 2020-04-03 to 2020-05-17 (27 trading days available)


Processing CSV: 1643rows [02:29, 10.98rows/s]


DONE — Output saved to:
D:\Trading Stratergies\Backtesting\50DMA_NIFTY.csv


In [5]:
#Latest_3days_EURINR_Ratio>50DMA_EURINR

import pandas as pd

# ----------- USER INPUTS -----------
input_file = r"D:\Trading Stratergies\Backtesting\50DMA_EURINR.csv"
output_file = r"D:\Trading Stratergies\Backtesting\Latest_3days_TotalPrice_GT_50DMA_EURINR.xlsx"
# -----------------------------------

print("Loading file...")
df = pd.read_csv(input_file, usecols=["DATE", "Combo", "EURINR_Ratio", "50DMA_EURINR"])

# Ensure DATE is datetime for proper sorting
df["DATE"] = pd.to_datetime(df["DATE"])

# -------- AUTO FETCH LATEST 3 DATES --------
latest_3_dates = sorted(df["DATE"].unique(), reverse=True)[:3]
print("Latest 3 Dates Detected:", latest_3_dates)

# Filter only those 3 dates
df = df[df["DATE"].isin(latest_3_dates)]

print("Applying condition: EURINR_Ratio > 50DMA_EURINR...")

# Keep only rows satisfying condition
df_valid = df[df["EURINR_Ratio"] > df["50DMA_EURINR"]]

# Count valid rows per combo
combo_counts = df_valid.groupby("Combo")["DATE"].nunique()

# Only keep combos passing ALL 3 dates
valid_combos = combo_counts[combo_counts == 3].index

final_df = df_valid[df_valid["Combo"].isin(valid_combos)]

# Sort: Combo wise + latest date first
final_df = final_df.sort_values(by=["Combo", "DATE"], ascending=[True, False])

# Convert DATE back to string for output clarity
final_df["DATE"] = final_df["DATE"].dt.strftime("%Y-%m-%d")

print("Saving output...")
final_df.to_excel(output_file, index=False)

print("\nDONE! Auto latest-3-days EURINR file created:")
print(output_file)


Loading file...
Latest 3 Dates Detected: [Timestamp('2020-05-15 00:00:00'), Timestamp('2020-05-14 00:00:00'), Timestamp('2020-05-13 00:00:00')]
Applying condition: EURINR_Ratio > 50DMA_EURINR...
Saving output...

DONE! Auto latest-3-days EURINR file created:
D:\Trading Stratergies\Backtesting\Latest_3days_TotalPrice_GT_50DMA_EURINR.xlsx


In [4]:
#Latest_3days_NIFTY_Ratio>50DMA_NIFTY

import pandas as pd

# ----------- USER INPUTS -----------
input_file = r"D:\Trading Stratergies\Backtesting\50DMA_NIFTY.csv"
output_file = r"D:\Trading Stratergies\Backtesting\Latest_3days_TotalPrice_GT_50DMA_NIFTY.xlsx"


print("Loading file...")
df = pd.read_csv(input_file, usecols=["DATE", "Combo", "NIFTY_Ratio", "50DMA_NIFTY"])

# Ensure DATE is datetime for proper sorting
df["DATE"] = pd.to_datetime(df["DATE"])

# -------- AUTO FETCH LATEST 3 DATES --------
latest_3_dates = sorted(df["DATE"].unique(), reverse=True)[:3]
print("Latest 3 Dates Detected:", latest_3_dates)

# Filter only those 3 dates
df = df[df["DATE"].isin(latest_3_dates)]

print("Applying condition: NIFTY_Ratio > 50DMA_NIFTY...")

# Keep only rows satisfying condition
df_valid = df[df["NIFTY_Ratio"] > df["50DMA_NIFTY"]]

# Count how many valid dates per combo
combo_counts = df_valid.groupby("Combo")["DATE"].nunique()

# Only combos present in ALL 3 latest dates
valid_combos = combo_counts[combo_counts == 3].index

final_df = df_valid[df_valid["Combo"].isin(valid_combos)]

# Sort output: Combo-wise & Latest date first
final_df = final_df.sort_values(by=["Combo", "DATE"], ascending=[True, False])

# Convert DATE back to string for Excel readability
final_df["DATE"] = final_df["DATE"].dt.strftime("%Y-%m-%d")

print("Saving output...")
final_df.to_excel(output_file, index=False)

print("\n DONE! Auto latest-3-days NIFTY file created:")
print(output_file)


Loading file...
Latest 3 Dates Detected: [Timestamp('2020-05-15 00:00:00'), Timestamp('2020-05-14 00:00:00'), Timestamp('2020-05-13 00:00:00')]
Applying condition: NIFTY_Ratio > 50DMA_NIFTY...
Saving output...

 DONE! Auto latest-3-days NIFTY file created:
D:\Trading Stratergies\Backtesting\Latest_3days_TotalPrice_GT_50DMA_NIFTY.xlsx


In [8]:
#Common_Combinations_EURINR_NIFTY

import pandas as pd

# ---------------- USER INPUTS ----------------
eurinr_file = r"D:\Trading Stratergies\Backtesting\Latest_3days_TotalPrice_GT_50DMA_EURINR.xlsx"
nifty_file = r"D:\Trading Stratergies\Backtesting\Latest_3days_TotalPrice_GT_50DMA_NIFTY.xlsx"
output_file = r"D:\Trading Stratergies\Backtesting\Common_Combinations_EURINR_NIFTY.xlsx"


print("Loading EURINR file...")
df_eur = pd.read_excel(eurinr_file, engine="openpyxl")

print("Loading NIFTY file...")
df_nifty = pd.read_excel(nifty_file, engine="openpyxl")

# Ensure DATE is datetime
df_eur["DATE"] = pd.to_datetime(df_eur["DATE"])
df_nifty["DATE"] = pd.to_datetime(df_nifty["DATE"])

# Drop duplicate combos if any
df_eur = df_eur.drop_duplicates(subset=["Combo", "DATE"])
df_nifty = df_nifty.drop_duplicates(subset=["Combo", "DATE"])

print("Finding common combos...")

# Merge on Combo AND DATE to get side-by-side values
merged_df = pd.merge(
    df_eur, 
    df_nifty, 
    on=["Combo", "DATE"], 
    how="inner", 
    suffixes=("_EURINR", "_NIFTY")
)

# Optional: sort by Combo & latest date first
merged_df = merged_df.sort_values(by=["Combo", "DATE"], ascending=[True, False])

# Convert DATE back to string for Excel readability
merged_df["DATE"] = merged_df["DATE"].dt.strftime("%Y-%m-%d")

print("Saving output...")
merged_df.to_excel(output_file, index=False)

print("\n DONE! Common combos with EURINR + NIFTY values saved at:")
print(output_file)


Loading EURINR file...
Loading NIFTY file...
Finding common combos...
Saving output...

 DONE! Common combos with EURINR + NIFTY values saved at:
D:\Trading Stratergies\Backtesting\Common_Combinations_EURINR_NIFTY.xlsx


In [9]:
#Ratio<=1.03*50DMA(latest_day)

import pandas as pd

# ---------------- INPUT / OUTPUT ----------------
input_file  = r"D:\Trading Stratergies\Backtesting\Common_Combinations_EURINR_NIFTY.xlsx"
output_file = r"D:\Trading Stratergies\Backtesting\Latest_day_Ratio_should_not_be_GT_1.03_times_50DMA.xlsx"


# Load data
df = pd.read_excel(input_file, engine="openpyxl")

# Ensure DATE is datetime
df["DATE"] = pd.to_datetime(df["DATE"])

# Detect latest date automatically
latest_date = df["DATE"].max()
print("Latest date detected:", latest_date.date())

# Filter rows for latest date
df_latest = df[df["DATE"] == latest_date]

# Apply vectorized condition
condition = (
    (df_latest["EURINR_Ratio"] <= 1.03 * df_latest["50DMA_EURINR"]) &
    (df_latest["NIFTY_Ratio"]  <= 1.03 * df_latest["50DMA_NIFTY"])
)

filtered_df = df_latest.loc[condition].reset_index(drop=True)

# Save output
filtered_df.to_excel(output_file, index=False, engine="openpyxl")

print("\nFiltering Completed")
print("Total rows for latest date :", len(df_latest))
print("Rows kept after filter    :", len(filtered_df))
print("Output saved to:", output_file)


Latest date detected: 2020-05-15

Filtering Completed
Total rows for latest date : 14042
Rows kept after filter    : 1140
Output saved to: D:\Trading Stratergies\Backtesting\Latest_day_Ratio_should_not_be_GT_1.03_times_50DMA.xlsx


In [10]:
#Top10_Highest_Average_Ratio

import pandas as pd

# ---------------- INPUT / OUTPUT ----------------
input_file  = r"D:\Trading Stratergies\Backtesting\Latest_day_Ratio_should_not_be_GT_1.03_times_50DMA.xlsx"
output_file = r"D:\Trading Stratergies\Backtesting\Top10_Highest_Average_Ratio.xlsx"

# Load data
df = pd.read_excel(input_file, engine="openpyxl")

# Compute average ratio per row (EURINR + NIFTY)
df["Avg_Ratio"] = (df["EURINR_Ratio"] + df["NIFTY_Ratio"]) / 2

# Group by Combo and calculate mean of Avg_Ratio
combo_avg = df.groupby("Combo")["Avg_Ratio"].mean().reset_index()

# Sort descending to get highest average ratios
combo_avg = combo_avg.sort_values(by="Avg_Ratio", ascending=False)

# Keep Top 10 combos
top10_combos = combo_avg.head(10)

# Optional: merge with original data to get all details for these combos
top10_details = df[df["Combo"].isin(top10_combos["Combo"])].reset_index(drop=True)

# Save to Excel
top10_details.to_excel(output_file, index=False, engine="openpyxl")

print("Top 10 highest average ratio combos saved to:", output_file)


Top 10 highest average ratio combos saved to: D:\Trading Stratergies\Backtesting\Top10_Highest_Average_Ratio.xlsx


In [11]:
#Average_Correlation_EURINR_NIFTY

import pandas as pd

# ---------------- FILE PATHS ----------------
file_corr = r"D:\Trading Stratergies\Stratergy 2 All Codes\4Stock_Combinations_Correlation.xlsx"    
file_data = r"D:\Trading Stratergies\Backtesting\Latest_day_Ratio_should_not_be_GT_1.03_times_50DMA.xlsx"
output_file = r"D:\Trading Stratergies\Backtesting\Average_Correlation_EURINR_NIFTY.xlsx"


def normalize_combo(combo):
    if pd.isna(combo):
        return combo
    stocks = [s.strip() for s in combo.split(",")]
    stocks_sorted = sorted(stocks)
    return ", ".join(stocks_sorted)


print("Loading files...")
df_corr = pd.read_excel(file_corr)
df_data = pd.read_excel(file_data)

# Normalize combo column in both files
print("Normalizing Combo format...")
df_corr["Combo_Normalized"] = df_corr["Combo"].apply(normalize_combo)
df_data["Combo_Normalized"] = df_data["Combo"].apply(normalize_combo)

print("Merging datasets...")
final_df = df_data.merge(
    df_corr[["Combo_Normalized", "Average_Realized_Correlation"]],
    on="Combo_Normalized",
    how="left"   # keeps all second file rows
)

# Optional: Drop rows where correlation not found
final_df = final_df.dropna(subset=["Average_Realized_Correlation"])

# Remove helper column
final_df.drop(columns=["Combo_Normalized"], inplace=True)

print("Saving output...")
final_df.to_excel(output_file, index=False)

print(" FIXED: Correlation values correctly applied.")


Loading files...
Normalizing Combo format...
Merging datasets...
Saving output...
 FIXED: Correlation values correctly applied.


In [12]:
#Top10_Lowest_Correlation

import pandas as pd

# -------- USER FILE PATHS --------
input_file = r"D:\Trading Stratergies\Backtesting\Average_Correlation_EURINR_NIFTY.xlsx"
output_file = r"D:\Trading Stratergies\Backtesting\Top10_Lowest_Correlation.xlsx"

print("Loading merged data file...")
df = pd.read_excel(input_file)

print("Sorting by lowest Average_Realized_Correlation...")

# Sort ascending = lowest correlation first
df_sorted = df.sort_values(by="Average_Realized_Correlation", ascending=True)

# Select top 10 lowest
top10_lowest = df_sorted.head(10)

print("Saving Top 10 lowest correlation combinations...")
top10_lowest.to_excel(output_file, index=False)

print(" Done! Top 10 Lowest Correlation file created.")


Loading merged data file...
Sorting by lowest Average_Realized_Correlation...
Saving Top 10 lowest correlation combinations...
 Done! Top 10 Lowest Correlation file created.


In [9]:
#Beta_Total_EURINR_NIFTY

import csv

beta_file = r"D:\Trading Stratergies\Stratergy 2 All Codes\beta_total.csv"
data_file = r"C:\Users\Swarupa\Downloads\Latest_day_Ratio_should_not_be_GT_1.03_times_50DMA.csv"
output_file = r"D:\Trading Stratergies\Backtesting\Beta_Sum_EURINR_NIFTY.csv"


def normalize_combo(combo):
    if not combo:
        return None
    combo = combo.replace("_Beta", "")
    parts = [c.strip() for c in combo.split(",") if c.strip()]
    parts.sort()
    return ",".join(parts)


print("Reading required combinations from second file...")

# STEP 1: Read ONLY combos needed 
required_combos = set()

with open(data_file, newline='', encoding="utf-8") as f:
    reader = csv.DictReader(f)
    if "Combo" not in reader.fieldnames:
        raise Exception("ERROR: 'Combo' column not found in second file")

    for row in reader:
        clean = normalize_combo(row.get("Combo"))
        if clean:
            required_combos.add(clean)

print(f"Total required combos: {len(required_combos)}")


# STEP 2: Build beta dictionary ONLY for required combos
print("Building Beta dictionary (streaming)...")
beta_dict = {}

with open(beta_file, newline='', encoding="utf-8") as f:
    reader = csv.DictReader(f)
    if "Combo" not in reader.fieldnames or "Beta_Sum" not in reader.fieldnames:
        raise Exception("ERROR: 'Combo' or 'Beta_Sum' column missing in beta file")

    for row in reader:
        clean = normalize_combo(row.get("Combo"))
        if clean in required_combos:
            beta_dict[clean] = row.get("Beta_Sum")

print("Beta dictionary ready.")


# STEP 3: Create final output
print("Writing output file...")

with open(data_file, newline='', encoding="utf-8") as f_in, \
     open(output_file, "w", newline='', encoding="utf-8") as f_out:

    reader = csv.DictReader(f_in)
    fieldnames = list(reader.fieldnames) + ["Beta_Sum"]
    writer = csv.DictWriter(f_out, fieldnames=fieldnames)
    writer.writeheader()

    row_count = 0

    for row in reader:
        clean = normalize_combo(row.get("Combo"))
        row["Beta_Sum"] = beta_dict.get(clean, "")
        writer.writerow(row)
        row_count += 1

print(f"COMPLETED SUCCESSFULLY - {row_count} rows written")
print(f" Output file: {output_file}")


Reading required combinations from second file...
Total required combos: 1140
Building Beta dictionary (streaming)...
Beta dictionary ready.
Writing output file...
COMPLETED SUCCESSFULLY - 1140 rows written
 Output file: D:\Trading Stratergies\Backtesting\Beta_Sum_EURINR_NIFTY.csv


In [10]:
#Top10_Lowest_Beta_Total

import pandas as pd

# -------- USER FILE PATHS --------
input_file = r"D:\Trading Stratergies\Backtesting\Beta_Sum_EURINR_NIFTY.csv"
output_file = r"D:\Trading Stratergies\Backtesting\Top10_Lowest_Beta_Sum.xlsx"

print("Loading merged data file...")
df = pd.read_csv(input_file)

print("Sorting by lowest Beta_Sum...")

# Sort ascending = lowest Beta_Sum first
df_sorted = df.sort_values(by="Beta_Sum", ascending=True)

# Select top 10 lowest Beta_Sum combos
top10_lowest = df_sorted.head(10)

print("Saving Top 10 lowest Beta_Sum combinations...")
top10_lowest.to_excel(output_file, index=False)

print(" Done! Top 10 Lowest Beta_Sum file created.")


Loading merged data file...
Sorting by lowest Beta_Sum...
Saving Top 10 lowest Beta_Sum combinations...
 Done! Top 10 Lowest Beta_Sum file created.
